In [1]:
#https://stackoverflow.com/questions/40536560/ipython-and-jupyter-autocomplete-not-working
%config Completer.use_jedi = False

from IPython.display import display, Markdown, HTML, Image

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import os
import sys
from pathlib import Path

import pandas as pd
import numpy as np

import logging
logging.basicConfig(stream=sys.stdout, format='',
                level=logging.INFO, datefmt=None)
logger = logging.getLogger('preprocessing')

In [2]:
def merge_by_concat(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

### Load dataset

In [3]:
#global
# variables
TARGET = 'sales'
TIME_HORIZON = 28
END_TRAIN = 1941 - 28  # total num of days is 1941, leave the last 28 out for testing purposes
DAY_COLUMN = 'd'

# load data
logger.info('Load Main Data')
DATA_PATH = Path('../data')
calendar_df = pd.read_csv(DATA_PATH / 'dataset/calendar.csv')
train_df = pd.read_csv(DATA_PATH / 'dataset/sales_train_evaluation.csv')
prices_df = pd.read_csv(DATA_PATH / 'dataset/sell_prices.csv')

Load Main Data


In [4]:
calendar_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [5]:
train_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [6]:
prices_df.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


### Create grid

In [7]:
# reformat train_df
# instead of days in a horizontal orientation, make it vertical
index_columns = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
grid_df = pd.melt(train_df,
                  id_vars=index_columns,
                  var_name=DAY_COLUMN,
                  value_name='sales')
grid_df.head()

,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,id,HOBBIES_1_001_CA_1_evaluation
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,id,HOBBIES_1_002_CA_1_evaluation
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,id,HOBBIES_1_003_CA_1_evaluation
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,id,HOBBIES_1_004_CA_1_evaluation
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,id,HOBBIES_1_005_CA_1_evaluation


### Create Holdout validation

In [8]:
last_training_day = END_TRAIN
end_val_day = END_TRAIN + TIME_HORIZON

grid_df['d'] = grid_df['d'].apply(lambda x: x[2:] if x[1] == '_' else -1).astype(np.int16)
grid_df['d'] = grid_df[grid_df.d > 0].d
holdout_df = grid_df[(grid_df['d'] > last_training_day) & (grid_df['d'] <= end_val_day)][index_columns + [DAY_COLUMN, TARGET]]
grid_df = grid_df[grid_df['d'] <= last_training_day]

holdout_df.reset_index(drop=True, inplace=True)
holdout_df['d'] = holdout_df['d'].apply(lambda x: 'd_' + str(x))
grid_df['d'] = grid_df['d'].apply(lambda x: 'd_' + str(x))
grid_df.head()

,item_id,dept_id,cat_id,store_id,state_id,d,sales
30490,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1.0,0
30491,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1.0,0
30492,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1.0,0
30493,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1.0,0
30494,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1.0,0


In [9]:
# add rows for test (not sure if needed)
logger.info('Adding test days')
add_grid = pd.DataFrame()
for i in range(1, TIME_HORIZON + 1):
    temp_df = train_df[index_columns]
    temp_df = temp_df.drop_duplicates()
    temp_df['d'] = 'd_' + str(END_TRAIN + i)
    temp_df[TARGET] = np.nan
    add_grid = pd.concat([add_grid, temp_df])

grid_df = pd.concat([grid_df, add_grid])
grid_df = grid_df.reset_index(drop=True)

# print(grid_df.tail())
del temp_df, add_grid, train_df

Adding test days


In [10]:
# Release dates
logger.info('Release')
release_df = prices_df.groupby(['store_id', 'item_id'])['wm_yr_wk'].agg(['min']).reset_index()
release_df.columns = ['store_id', 'item_id', 'release']
# print(release_df.head())

grid_df = merge_by_concat(grid_df, release_df, ['store_id', 'item_id'])  # match release date w each product (store_id, item_id)
# print(grid_df.head(20))
idx = calendar_df.index.values.tolist()
d = ['d_' + str(x + 1) for x in idx]
calendar_df['d'] = d
# print(calendar_df.head())
grid_df = merge_by_concat(grid_df, calendar_df[['wm_yr_wk', 'd']], ['d'])  # match day number w wm_yr_wk
# print(grid_df.head(20))
grid_df = grid_df[grid_df['wm_yr_wk'] >= grid_df['release']].reset_index(drop=True)  # for each product only keep if day is after release date -> delete useless rows
# print(grid_df.head(20))

Release


### Add calendar

In [11]:
# calendar_cols = ['date', 'd', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI']
calendar_cols = ['date', 'd']
grid_df = grid_df.merge(calendar_df[calendar_cols], on="d", how="left")
holdout_df = holdout_df.merge(calendar_df[calendar_cols], on="d", how="left")

In [13]:
grid_df.info()
grid_df.head()
grid_df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 853720 entries, 0 to 853719
Data columns (total 10 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   item_id   853720 non-null  object 
 1   dept_id   853720 non-null  object 
 2   cat_id    853720 non-null  object 
 3   store_id  853720 non-null  object 
 4   state_id  853720 non-null  object 
 5   d         853720 non-null  object 
 6   sales     0 non-null       object 
 7   release   853720 non-null  int64  
 8   wm_yr_wk  853720 non-null  float64
 9   date      853720 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 71.6+ MB


,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,wm_yr_wk,date
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,NaN,11325,11613.0,2016-04-25
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,NaN,11121,11613.0,2016-04-25
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,NaN,11401,11613.0,2016-04-25
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,NaN,11106,11613.0,2016-04-25
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1914,NaN,11117,11613.0,2016-04-25


,release,wm_yr_wk
count,853720.000000,853720.000000
mean,11200.380912,11614.785714
std,122.571718,1.205854
min,11101.000000,11613.000000
25%,11101.000000,11614.000000
50%,11123.000000,11615.000000
75%,11306.000000,11616.000000
max,11603.000000,11617.000000


In [14]:
holdout_df.info()
holdout_df.head()
holdout_df.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 853720 entries, 0 to 853719
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   item_id   853720 non-null  object
 1   dept_id   853720 non-null  object
 2   cat_id    853720 non-null  object
 3   store_id  853720 non-null  object
 4   state_id  853720 non-null  object
 5   d         853720 non-null  object
 6   sales     853720 non-null  object
 7   date      0 non-null       object
dtypes: object(8)
memory usage: 58.6+ MB


,item_id,dept_id,cat_id,store_id,state_id,d,sales,date
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1914.0,0,NaN
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1914.0,0,NaN
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1914.0,0,NaN
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1914.0,0,NaN
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1914.0,1,NaN


,item_id,dept_id,cat_id,store_id,state_id,d,sales,date
count,853720,853720,853720,853720,853720,853720,853720,0
unique,3049,7,3,10,3,28,132,0
top,HOBBIES_1_407,FOODS_3,FOODS,TX_2,CA,d_1934.0,0,NaN
freq,280,230440,402360,85372,341488,30490,464725,NaN


### Save preprocessing

In [15]:
holdout_df.to_csv(f'{DATA_PATH}/holdout.csv')
grid_df.to_csv(f'{DATA_PATH}/preprocessed.csv')